# Initializing

In [45]:
from scipy.signal import convolve2d
from matplotlib import pyplot as plt
import numpy as np
import cv2
from PIL import Image
from numpy import asarray
from pyperclip import copy
from pyzbar.pyzbar import decode
import urllib.request
import requests
from skimage.metrics import structural_similarity as ssim
%matplotlib inline

In [46]:
def BarcodeReader(image):
    img = cv2.imread(image)
    detectedBarcodes = decode(img)
    if not detectedBarcodes:
        print("Barcode Not Detected or your barcode is blank/corrupted!")
    else:
        for barcode in detectedBarcodes: 
            (x, y, w, h) = barcode.rect
            cv2.rectangle(img, (x-10, y-10),
                          (x + w+10, y + h+10),
                          (255, 0, 0), 2)
            if barcode.data!="":
                return str(int(barcode.data))
                 
def ISBNcover(isbn):
    return'https://d1w7fb2mkkr3kw.cloudfront.net/assets/images/book/lrg/' + str(int(isbn))[:4] + '/' + str(int(isbn))[4:8] + '/' + str(int(isbn)) + '.jpg'

def save(weblink):
    urllib.request.urlretrieve(weblink, "coverphoto.jpg")

def pricegenerator(isbn):
    page = requests.get('https://www.bookdepository.com/search/' + str(isbn))
    htmlcode1 = str(page.text)
    start = htmlcode1.find('<p class="list-price">')
    htmlcode2 = htmlcode1[start+22:].strip()[15:].strip()
    end = htmlcode2.find('</p>')
    htmlcodesnippet = htmlcode2[:end]
    return htmlcodesnippet

In [55]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
import cv2
from PIL import Image
def crop(image='coverphotoused',alpha= 2):
    image = cv2.imread(image + '.jpg')
    beta = 10
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    cv2.imwrite('coverphotousedl.jpg', adjusted)
    image= cv2.imread('coverphotousedl.jpg')
    original_image= image
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    edges= cv2.Canny(gray, 50,200)
    contours, hierarchy= cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
    for (i,c) in enumerate(sorted_contours):
        x,y,w,h= cv2.boundingRect(c)
        cropped_contour= original_image[y:y+h, x:x+w]
        image_name= "coverphotousedc.jpg"
        cv2.imwrite(image_name, cropped_contour)
        readimage= cv2.imread(image_name)
def resize(filename):
    img = Image.open(filename+'.jpg')
    img = img.resize((100,148))
    img.save(filename + '.jpg')

def grayscale(filename):
    img_rgb = Image.open(filename+'.jpg')
    img_gray = img_rgb.convert('L')
    img_gray.save(filename+'g.jpg')
def runcropscale(image = 'coverphotoused'):
    crop(image)
    resize('coverphoto')
    resize('coverphotousedc')
    grayscale('coverphoto')
    grayscale('coverphotousedc')
    gi1 = cv2.imread("coverphotog.jpg")
    gi2 = cv2.imread("coverphotousedcg.jpg")
    return ssim(gi1, gi2, channel_axis=2)

In [48]:
def checkimg(coverphoto = 'coverphoto', nia='nia.jpg'):
    coverimg = Image.open(coverphoto + '.jpg')
    niaimg = Image.open(nia)
    cover = asarray(coverimg)
    noimageava = asarray(niaimg)
    if cover.all() == noimageava.all():
        return False
    return True

def pricetag(pc,price):
    override = input(f'Automatic price generated: {price} \n Manual Override T/F ')
    if override.lower() == 't':
        price = input('New price: ')
    else:
        price = str(price * pc)
    copy(price)
    return price

In [49]:
def full(used = 'coverphotoused', barcode = 'bb', testing = False):
    if not testing:
        save(ISBNcover(BarcodeReader('bb.jpg')))
    price = pricegenerator(BarcodeReader('bb.jpg'))
    if not checkimg():
        return 'Not avaliable'
    pc = float(runcropscale(used))
    if testing:
        print(pc)
    pricetag(int(pc), price)


In [53]:
save(ISBNcover('9781409329800'))

In [54]:
full(testing = True)

0.07840083813158158
Automatic price generated: 117.56 
 Manual Override T/F f
